In [1]:
import pandas as pd
import numpy as np
import ast
import datetime
from math import radians, cos, sin, asin, sqrt

In [2]:
usecols = ['CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 
           'TAXI_ID', 'TIMESTAMP', 
           'POLYLINE', 'dist_perc', 'start', 'stop']

df = pd.read_csv('train_tratado_outliers.csv', usecols=usecols, parse_dates=True, nrows=100000)

# cabeçalho minusculo
df.columns = df.columns.map(lambda x: x.lower())
#conversão da data de obj para datetime
df.timestamp = pd.to_datetime(df.timestamp)

In [3]:
df.head()

,call_type,origin_call,origin_stand,taxi_id,timestamp,polyline,dist_perc,start,stop
0,C,NaN,NaN,20000589,2013-07-01 00:00:58,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",2.23,"[-8.618643, 41.141412]","[-8.630838, 41.154489]"
1,B,NaN,7.0,20000596,2013-07-01 00:08:23,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",3.46,"[-8.639847, 41.159826]","[-8.66574, 41.170671]"
2,C,NaN,NaN,20000320,2013-07-01 00:02:31,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",17.63,"[-8.612964, 41.140359]","[-8.61597, 41.14053]"
3,C,NaN,NaN,20000520,2013-07-01 00:00:54,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",7.97,"[-8.574678, 41.151951]","[-8.607996, 41.142915]"
4,C,NaN,NaN,20000337,2013-07-01 00:04:51,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",4.81,"[-8.645994, 41.18049]","[-8.687268, 41.178087]"


Working with polyline (get first, last and penultimate)

In [4]:
del df['start']
del df['stop']

In [5]:
def points(x):
    try:
        
        global cont;
        
        line = np.array(ast.literal_eval(x))
        
        if line.size>2:
            return line[0][1], line[0][0], line[-2][1], line[-2][0], line[-1][1], line[-1][0]
        
        return 0,0,0,0,0,0
    except Exception as e:
        #print(e)
        None
   #finally:
   #    if cont%10000 ==0: print(datetime.datetime.now(), cont)

In [6]:
cont=0
df['sta_lat'],df['sta_lon'],df['pen_lat'],df['pen_lon'],df['sto_lat'],df['sto_lon'] = zip(*df['polyline'].apply(points))

In [7]:
del df['polyline']

Treating timestamp

In [8]:
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

In [9]:
#df['cat_hour'] = pd.cut(df.timestamp.dt.hour, [-1, 8, 16, 23], labels=[0,1,2])
df['hour'] = df.timestamp.dt.hour

In [10]:
del df['timestamp']

In [11]:
df.head()

,call_type,origin_call,origin_stand,taxi_id,dist_perc,sta_lat,sta_lon,pen_lat,pen_lon,sto_lat,sto_lon,month,day,hour
0,C,NaN,NaN,20000589,2.23,41.141412,-8.618643,41.154498,-8.630829,41.154489,-8.630838,7,1,0
1,B,NaN,7.0,20000596,3.46,41.159826,-8.639847,41.170635,-8.665767,41.170671,-8.665740,7,1,0
2,C,NaN,NaN,20000320,17.63,41.140359,-8.612964,41.140863,-8.617365,41.140530,-8.615970,7,1,0
3,C,NaN,NaN,20000520,7.97,41.151951,-8.574678,41.142915,-8.608005,41.142915,-8.607996,7,1,0
4,C,NaN,NaN,20000337,4.81,41.180490,-8.645994,41.178078,-8.687259,41.178087,-8.687268,7,1,0


Tratando call_type

In [12]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

df['call_type'] = preprocessing.LabelEncoder().fit_transform(df['call_type'])

enc = OneHotEncoder().fit_transform(df[['call_type']])

del df['call_type']

df['call_type_a'] = enc.toarray()[:,0]
df['call_type_b'] = enc.toarray()[:,1]
df['call_type_c'] = enc.toarray()[:,2]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
origin_call     20962 non-null float64
origin_stand    49311 non-null float64
taxi_id         100000 non-null int64
dist_perc       100000 non-null float64
sta_lat         100000 non-null float64
sta_lon         100000 non-null float64
pen_lat         100000 non-null float64
pen_lon         100000 non-null float64
sto_lat         100000 non-null float64
sto_lon         100000 non-null float64
month           100000 non-null int64
day             100000 non-null int64
hour            100000 non-null int64
call_type_a     100000 non-null float64
call_type_b     100000 non-null float64
call_type_c     100000 non-null float64
dtypes: float64(12), int64(4)
memory usage: 12.2 MB


Filling NA with -1 (all columns)

In [14]:
df.fillna(-1, inplace=True)

In [15]:
df.head()

,origin_call,origin_stand,taxi_id,dist_perc,sta_lat,sta_lon,pen_lat,pen_lon,sto_lat,sto_lon,month,day,hour,call_type_a,call_type_b,call_type_c
0,-1.0,-1.0,20000589,2.23,41.141412,-8.618643,41.154498,-8.630829,41.154489,-8.630838,7,1,0,0.0,0.0,1.0
1,-1.0,7.0,20000596,3.46,41.159826,-8.639847,41.170635,-8.665767,41.170671,-8.665740,7,1,0,0.0,1.0,0.0
2,-1.0,-1.0,20000320,17.63,41.140359,-8.612964,41.140863,-8.617365,41.140530,-8.615970,7,1,0,0.0,0.0,1.0
3,-1.0,-1.0,20000520,7.97,41.151951,-8.574678,41.142915,-8.608005,41.142915,-8.607996,7,1,0,0.0,0.0,1.0
4,-1.0,-1.0,20000337,4.81,41.180490,-8.645994,41.178078,-8.687259,41.178087,-8.687268,7,1,0,0.0,0.0,1.0


## Start ML processing

Separating X and y

In [16]:
y = df[['sto_lat', 'sto_lon']]

In [17]:
X = df.copy().drop(['sto_lat', 'sto_lon'], axis=1)

Normalizing with Min Max scaler (data is not in normal distribution)

In [18]:
from sklearn.preprocessing import MinMaxScaler

X_t = MinMaxScaler().fit_transform(X)

Split train/test

In [19]:
from sklearn.model_selection import train_test_split

X1, X2, y1, y2 = train_test_split(X_t, y, test_size=0.33, random_state=42)

Testing some regressors

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

from sklearn.multioutput import MultiOutputRegressor

In [118]:
algs =[('linear', LinearRegression()),
       ('ridge', Ridge(random_state=99)),
       ('lasso', Lasso(random_state=99)), #lasso 142.7065081365015 561.2795889474457
       ('lsvr', LinearSVR(random_state=99)),
       ('KNN reg', KNeighborsRegressor()), #KNN reg 3.3861972620770264 3.5182287757352873
       ('gboost', GradientBoostingRegressor(random_state=99)),
       ('ada boost', AdaBoostRegressor(random_state=99)),
       ('rnd forest', RandomForestRegressor(random_state=99)),
       ('xgboost', XGBRegressor(random_state=99, n_jobs=-1))
       ]

for name, alg in algs:
    try:
        alg.fit(X1, y1)
        preds = alg.predict(X2)

        result = y2.copy()
        result['pred_lat'] = preds[:,0]
        result['pred_lon'] = preds[:,1]
        hav_dist = result.apply(lambda x: haversine(x.sto_lon, x.sto_lat, x.pred_lon, x.pred_lat) ,axis=1) 

        print('normal',name, hav_dist.mean(), hav_dist.std())
    except: # se o algoritmo nao possui suporte nativo para multioutput
        mo = MultiOutputRegressor(alg, n_jobs=-1).fit(X1, y1)
        preds = mo.predict(X2)
        
        result = y2.copy()
        result['pred_lat'] = preds[:,0]
        result['pred_lon'] = preds[:,1]
        hav_dist = result.apply(lambda x: haversine(x.sto_lon, x.sto_lat, x.pred_lon, x.pred_lat) ,axis=1) 
        
        print('multi',name, hav_dist.mean(), hav_dist.std())

normal linear 0.04720105786224005 0.19469234340305402
normal ridge 3.1070450960957117 4.1599047429184735
normal lasso 142.7065081365015 561.2795889474457
multi lsvr 0.5301121250279166 0.7767335127405629
normal KNN reg 3.3861972620770264 3.5182287757352873
multi gboost 0.10519601382980781 0.282520022430942
multi ada boost 2.4819894066456234 2.4932608615782255
normal rnd forest 0.08761489620151998 0.7359845836661133
multi xgboost 0.17196969804172219 0.3216972494653177


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000000DEAC5754E0>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [120]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    #dlon = lon2 - lon1 
    #dlat = lat2 - lat1 
    a = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

Linear regression(!), Random Forest and Gradient Boost was best test models. Let's try tune them with GridSearch:

In [21]:
def haversine2(y, y_pred):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1 = y
    lon2, lat2 = y_pred
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    #dlon = lon2 - lon1 
    #dlat = lat2 - lat1 
    a = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [22]:
from sklearn.metrics import make_scorer

haversine_scorer = make_scorer(haversine2, greater_is_better=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid={
    'n_estiamtors':[100, 200],
    'max_depth': [5, 10]
}

gs = GridSearchCV(RandomForestRegressor(random_state=99), param_grid=param_grid, n_jobs=-1, scoring=haversine_scorer, cv=10, verbose=2)
gs.fit(X1, y1)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
